# Notebook 03: Model Training and Evaluation

## Project Overview

This notebook trains and evaluates machine learning models to predict Apple call option prices.

**Objectives:**
- Load clean ML-ready dataset
- Perform dimensionality reduction analysis (PCA)
- Split data into train/validation/test sets (stratified by maturity)
- Train baseline and advanced models with hyperparameter tuning
- Evaluate performance with multiple metrics
- Analyze errors and model behavior
- Save best model for future use

**Models tested:**
1. Linear Regression (baseline)
2. Random Forest (with Grid Search)
3. XGBoost (with Grid Search)

**Target:** Predict C_LAST (call option price)

**References:**
- Chen & Guestrin (2016): XGBoost - A Scalable Tree Boosting System
- Breiman (2001): Random Forests
- Hutchinson et al. (1994): Nonparametric Option Pricing via Learning Networks

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV,
    learning_curve
)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
plt.style.use('seaborn-v0_8-whitegrid')

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
FIGS_DIR = PROJECT_ROOT / "figs"
MODELS_DIR = PROJECT_ROOT / "models"

FIGS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

ML_DATA_FILE = DATA_PROCESSED_DIR / "AAPL_ML_READY.csv"

In [ ]:
df = pd.read_csv(ML_DATA_FILE)
print(f"Dataset loaded: {df.shape[0]:,} rows x {df.shape[1]} columns")
df.head()

In [ ]:
print(f"Missing values: {df.isna().sum().sum()}")
print(f"Duplicates: {df.duplicated().sum()}")
print(f"Target range: [{df['C_LAST'].min():.2f}, {df['C_LAST'].max():.2f}] USD")
df.describe()

## 2. Correlation Analysis

In [ ]:
numeric_cols = ['UNDERLYING_LAST', 'STRIKE', 'DTE', 'LOG_DTE', 'LOG_MONEYNESS', 'C_IV', 'C_LAST']
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix, 
    annot=True, 
    fmt='.2f', 
    cmap='RdBu_r', 
    center=0,
    mask=mask,
    square=True,
    linewidths=0.5,
    cbar_kws={'label': 'Correlation Coefficient'}
)
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGS_DIR / "correlation_heatmap.png", dpi=120, bbox_inches='tight')
plt.show()

print("\nCorrelation with target (C_LAST):")
print(corr_matrix['C_LAST'].sort_values(ascending=False))

## 3. Dimensionality Reduction Analysis (PCA)

We perform PCA to understand the variance structure of our features.
This helps identify if dimensionality reduction could improve model performance.

In [ ]:
feature_cols = ['UNDERLYING_LAST', 'STRIKE', 'DTE', 'LOG_DTE', 'LOG_MONEYNESS', 'C_IV']
X_pca = df[feature_cols].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pca)

pca = PCA()
pca.fit(X_scaled)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("PCA Explained Variance:")
for i, (var, cum) in enumerate(zip(explained_variance, cumulative_variance)):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%) | Cumulative: {cum:.4f} ({cum*100:.2f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

components = range(1, len(explained_variance) + 1)
axes[0].bar(components, explained_variance, alpha=0.7, color='steelblue', label='Individual')
axes[0].plot(components, cumulative_variance, 'ro-', label='Cumulative')
axes[0].axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA Scree Plot', fontweight='bold')
axes[0].legend()
axes[0].set_xticks(components)
axes[0].grid(alpha=0.3)

loadings = pd.DataFrame(
    pca.components_[:2].T,
    columns=['PC1', 'PC2'],
    index=feature_cols
)
loadings.plot(kind='barh', ax=axes[1], color=['steelblue', 'coral'])
axes[1].set_xlabel('Loading')
axes[1].set_title('Feature Loadings (PC1 & PC2)', fontweight='bold')
axes[1].axvline(x=0, color='black', linewidth=0.5)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGS_DIR / "pca_analysis.png", dpi=120)
plt.show()

n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nConclusion: {n_components_95} components needed to explain 95% of variance.")

## 4. Train/Validation/Test Split

We split the data into three sets:
- **Train (70%):** Used to train models
- **Validation (15%):** Used to tune hyperparameters
- **Test (15%):** Used for final evaluation (never seen during training)

**Important:** We use **stratified splitting** by maturity (DTE_BUCKET) to ensure all sets have similar distributions of short-term and long-term options.

In [ ]:
feature_cols = ['UNDERLYING_LAST', 'STRIKE', 'DTE', 'LOG_DTE', 'LOG_MONEYNESS', 'C_IV']
X = df[feature_cols]
y = df['C_LAST']

X_train, X_temp, y_train, y_temp, bucket_train, bucket_temp = train_test_split(
    X, y, df['DTE_BUCKET'],
    test_size=0.30,
    random_state=42,
    stratify=df['DTE_BUCKET']
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=bucket_temp
)

print(f"Train size:      {X_train.shape[0]:,} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation size: {X_val.shape[0]:,} ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test size:       {X_test.shape[0]:,} ({X_test.shape[0]/len(X)*100:.1f}%)")

## 5. Helper Functions

In [ ]:
def evaluate_model(y_true, y_pred, dataset_name=""):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"{dataset_name} Metrics:")
    print(f"  MAE:  {mae:.2f} USD")
    print(f"  RMSE: {rmse:.2f} USD")
    print(f"  R²:   {r2:.4f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}


def plot_learning_curve(estimator, X, y, title, cv=5, n_jobs=-1):
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='neg_mean_absolute_error'
    )
    
    train_scores_mean = -train_scores.mean(axis=1)
    train_scores_std = train_scores.std(axis=1)
    val_scores_mean = -val_scores.mean(axis=1)
    val_scores_std = val_scores.std(axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.fill_between(train_sizes, 
                     train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, 
                     alpha=0.1, color='blue')
    plt.fill_between(train_sizes, 
                     val_scores_mean - val_scores_std,
                     val_scores_mean + val_scores_std, 
                     alpha=0.1, color='orange')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training MAE')
    plt.plot(train_sizes, val_scores_mean, 'o-', color='orange', label='Validation MAE')
    
    plt.xlabel('Training Set Size')
    plt.ylabel('Mean Absolute Error (USD)')
    plt.title(f'Learning Curve: {title}', fontweight='bold')
    plt.legend(loc='best')
    plt.grid(alpha=0.3)
    
    return train_sizes, train_scores_mean, val_scores_mean

## 6. Model 1: Linear Regression (Baseline)

We start with the simplest model to establish a baseline performance.

In [ ]:
print("Training Linear Regression...")
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr_train = lr.predict(X_train)
y_pred_lr_val = lr.predict(X_val)
y_pred_lr_test = lr.predict(X_test)

print("\n" + "="*60)
print("LINEAR REGRESSION RESULTS")
print("="*60)
lr_train_metrics = evaluate_model(y_train, y_pred_lr_train, "Train")
print()
lr_val_metrics = evaluate_model(y_val, y_pred_lr_val, "Validation")
print()
lr_test_metrics = evaluate_model(y_test, y_pred_lr_test, "Test")

all_results = [{'Model': 'Linear Regression', **lr_test_metrics}]

## 7. Model 2: Random Forest with Grid Search

Random Forest can capture non-linear relationships and interactions between features.
We use **Grid Search** to find the optimal hyperparameters.

In [ ]:
print("Running Grid Search for Random Forest...")

GRID_SEARCH_SAMPLE_SIZE = 100000
np.random.seed(42)
sample_idx = np.random.choice(len(X_train), size=min(GRID_SEARCH_SAMPLE_SIZE, len(X_train)), replace=False)
X_train_sample = X_train.iloc[sample_idx]
y_train_sample = y_train.iloc[sample_idx]
print(f"Grid Search using {len(X_train_sample):,} samples (from {len(X_train):,} total)")

rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_leaf': [5, 10]
}

rf_base = RandomForestRegressor(n_jobs=-1, random_state=42)

rf_grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=rf_param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

rf_grid_search.fit(X_train_sample, y_train_sample)

print("\n" + "="*60)
print("RANDOM FOREST GRID SEARCH RESULTS")
print("="*60)
print(f"Best parameters: {rf_grid_search.best_params_}")
print(f"Best CV MAE (on sample): {-rf_grid_search.best_score_:.2f} USD")

In [ ]:
rf_best = rf_grid_search.best_estimator_

y_pred_rf_train = rf_best.predict(X_train)
y_pred_rf_val = rf_best.predict(X_val)
y_pred_rf_test = rf_best.predict(X_test)

print("Random Forest Performance (Best Hyperparameters):")
print("-" * 50)
rf_train_metrics = evaluate_model(y_train, y_pred_rf_train, "Train")
print()
rf_val_metrics = evaluate_model(y_val, y_pred_rf_val, "Validation")
print()
rf_test_metrics = evaluate_model(y_test, y_pred_rf_test, "Test")

all_results.append({'Model': 'Random Forest', **rf_test_metrics})

In [ ]:
rf_cv_results = pd.DataFrame(rf_grid_search.cv_results_)
rf_cv_results['MAE'] = -rf_cv_results['mean_test_score']

pivot_rf = rf_cv_results.pivot_table(
    values='MAE',
    index='param_max_depth',
    columns='param_n_estimators',
    aggfunc='mean'
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_rf, annot=True, fmt='.2f', cmap='YlOrRd_r', cbar_kws={'label': 'MAE (USD)'})
plt.title('Random Forest Grid Search: MAE by Hyperparameters', fontweight='bold')
plt.xlabel('n_estimators')
plt.ylabel('max_depth')
plt.tight_layout()
plt.savefig(FIGS_DIR / "rf_grid_search_heatmap.png", dpi=120)
plt.show()

## 8. Model 3: XGBoost with Grid Search

XGBoost is a state-of-the-art gradient boosting algorithm.
We use Grid Search combined with early stopping for optimal performance.

In [ ]:
print("Running Grid Search for XGBoost...")
print(f"Grid Search using {len(X_train_sample):,} samples")

xgb_param_grid = {
    'max_depth': [6, 10],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [200, 500]
}

xgb_base = XGBRegressor(
    objective='reg:squarederror',
    subsample=0.8,
    n_jobs=-1,
    random_state=42
)

xgb_grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=xgb_param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

xgb_grid_search.fit(X_train_sample, y_train_sample)

print("\n" + "="*60)
print("XGBOOST GRID SEARCH RESULTS")
print("="*60)
print(f"Best parameters: {xgb_grid_search.best_params_}")
print(f"Best CV MAE (on sample): {-xgb_grid_search.best_score_:.2f} USD")

In [ ]:
best_params = xgb_grid_search.best_params_.copy()
if 'n_estimators' in best_params:
    del best_params['n_estimators']

xgb_final = XGBRegressor(
    **best_params,
    n_estimators=2000,
    early_stopping_rounds=50,
    objective='reg:squarederror',
    subsample=0.8,
    n_jobs=-1,
    random_state=42
)

xgb_final.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print(f"Training stopped at iteration: {xgb_final.best_iteration}")
print(f"Best validation RMSE: {xgb_final.best_score:.4f}")

In [ ]:
y_pred_xgb_train = xgb_final.predict(X_train)
y_pred_xgb_val = xgb_final.predict(X_val)
y_pred_xgb_test = xgb_final.predict(X_test)

print("XGBoost Performance (Best Hyperparameters + Early Stopping):")
print("-" * 50)
xgb_train_metrics = evaluate_model(y_train, y_pred_xgb_train, "Train")
print()
xgb_val_metrics = evaluate_model(y_val, y_pred_xgb_val, "Validation")
print()
xgb_test_metrics = evaluate_model(y_test, y_pred_xgb_test, "Test")

all_results.append({'Model': 'XGBoost', **xgb_test_metrics})

In [ ]:
xgb_cv_results = pd.DataFrame(xgb_grid_search.cv_results_)
xgb_cv_results['MAE'] = -xgb_cv_results['mean_test_score']

pivot_xgb = xgb_cv_results.pivot_table(
    values='MAE',
    index='param_max_depth',
    columns='param_learning_rate',
    aggfunc='mean'
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_xgb, annot=True, fmt='.2f', cmap='YlOrRd_r', cbar_kws={'label': 'MAE (USD)'})
plt.title('XGBoost Grid Search: MAE by Hyperparameters', fontweight='bold')
plt.xlabel('learning_rate')
plt.ylabel('max_depth')
plt.tight_layout()
plt.savefig(FIGS_DIR / "xgb_grid_search_heatmap.png", dpi=120)
plt.show()

## 9. Model Comparison

In [ ]:
results_df = pd.DataFrame(all_results)

print("="*70)
print("MODEL COMPARISON ON TEST SET")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

best_model = results_df.loc[results_df['MAE'].idxmin(), 'Model']
print(f"\nBest Model: {best_model}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['MAE', 'RMSE', 'R2']
colors = ['coral', 'steelblue', 'seagreen']
titles = ['Mean Absolute Error (Lower is Better)', 
          'Root Mean Squared Error (Lower is Better)', 
          'R² Score (Higher is Better)']

for i, (metric, color, title) in enumerate(zip(metrics, colors, titles)):
    bars = axes[i].bar(results_df['Model'], results_df[metric], color=color, alpha=0.8, edgecolor='black')
    axes[i].set_title(title, fontweight='bold')
    axes[i].set_ylabel(metric)
    axes[i].tick_params(axis='x', rotation=15)
    axes[i].grid(axis='y', alpha=0.3)
    
    for bar, val in zip(bars, results_df[metric]):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                     f'{val:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(FIGS_DIR / "Model_Comparison.png", dpi=120)
plt.show()

## 10. Predicted vs Actual Plot

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sample_idx = np.random.choice(len(y_test), size=min(10000, len(y_test)), replace=False)
y_test_sample = y_test.iloc[sample_idx]

predictions = [
    (y_pred_lr_test[sample_idx], 'Linear Regression', 'coral'),
    (y_pred_rf_test[sample_idx], 'Random Forest', 'steelblue'),
    (y_pred_xgb_test[sample_idx], 'XGBoost', 'seagreen')
]

for i, (y_pred, title, color) in enumerate(predictions):
    axes[i].scatter(y_test_sample, y_pred, alpha=0.3, s=10, color=color)
    
    max_val = max(y_test_sample.max(), y_pred.max())
    axes[i].plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    axes[i].set_xlabel('Actual Price (USD)')
    axes[i].set_ylabel('Predicted Price (USD)')
    axes[i].set_title(f'{title}\nPredicted vs Actual', fontweight='bold')
    axes[i].legend()
    axes[i].grid(alpha=0.3)
    axes[i].set_xlim(0, 150)
    axes[i].set_ylim(0, 150)

plt.tight_layout()
plt.savefig(FIGS_DIR / "predicted_vs_actual.png", dpi=120)
plt.show()

## 11. Learning Curves (Overfitting/Underfitting Diagnosis)

In [ ]:
print("Generating learning curves...")

LEARNING_CURVE_SAMPLE_SIZE = 30000
X_lc_sample = X_train.sample(n=LEARNING_CURVE_SAMPLE_SIZE, random_state=42)
y_lc_sample = y_train.loc[X_lc_sample.index]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = [
    (LinearRegression(), 'Linear Regression'),
    (RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42), 'Random Forest'),
    (XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, n_jobs=-1, random_state=42), 'XGBoost')
]

for i, (model, title) in enumerate(models):
    train_sizes, train_scores, val_scores = learning_curve(
        model, X_lc_sample, y_lc_sample,
        cv=3,
        n_jobs=-1,
        train_sizes=np.linspace(0.2, 1.0, 5),
        scoring='neg_mean_absolute_error'
    )
    
    train_mean = -train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = -val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)
    
    axes[i].fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    axes[i].fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='orange')
    axes[i].plot(train_sizes, train_mean, 'o-', color='blue', label='Training')
    axes[i].plot(train_sizes, val_mean, 'o-', color='orange', label='Validation')
    
    axes[i].set_xlabel('Training Set Size')
    axes[i].set_ylabel('MAE (USD)')
    axes[i].set_title(f'Learning Curve: {title}', fontweight='bold')
    axes[i].legend(loc='best')
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGS_DIR / "learning_curves.png", dpi=120)
plt.show()

## 12. Error Analysis

In [ ]:
residuals = y_test - y_pred_xgb_test

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(y_pred_xgb_test, residuals, alpha=0.3, s=1, color='blue')
axes[0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Price (USD)')
axes[0].set_ylabel('Residuals (Actual - Predicted)')
axes[0].set_title('Residuals vs Predicted Values', fontweight='bold')
axes[0].grid(alpha=0.3)

axes[1].hist(residuals, bins=100, alpha=0.7, color='purple', edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].axvline(residuals.mean(), color='green', linestyle='-', linewidth=2, label=f'Mean: {residuals.mean():.2f}')
axes[1].set_xlabel('Residuals (USD)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGS_DIR / "residuals_analysis.png", dpi=120)
plt.show()

In [ ]:
test_analysis = X_test.copy()
test_analysis['C_LAST_TRUE'] = y_test.values
test_analysis['C_LAST_PRED'] = y_pred_xgb_test
test_analysis['ABS_ERROR'] = np.abs(test_analysis['C_LAST_TRUE'] - test_analysis['C_LAST_PRED'])

bins = np.linspace(test_analysis['LOG_MONEYNESS'].min(), test_analysis['LOG_MONEYNESS'].max(), 30)
test_analysis['MONEY_BUCKET'] = pd.cut(test_analysis['LOG_MONEYNESS'], bins=bins)

error_by_bucket = test_analysis.groupby('MONEY_BUCKET', observed=True).agg({
    'ABS_ERROR': 'mean',
    'LOG_MONEYNESS': 'mean'
}).dropna()

plt.figure(figsize=(12, 6))
plt.plot(error_by_bucket['LOG_MONEYNESS'], error_by_bucket['ABS_ERROR'], 
         marker='o', linewidth=2, markersize=6, color='darkblue')
plt.axvline(0, color='red', linestyle='--', linewidth=2, label='ATM (log(S/K)=0)')
plt.fill_between(error_by_bucket['LOG_MONEYNESS'].values, 0, 
                 error_by_bucket['ABS_ERROR'].values, alpha=0.2, color='blue')
plt.xlabel('Log-Moneyness = log(S/K)', fontsize=12)
plt.ylabel('Mean Absolute Error (USD)', fontsize=12)
plt.title('XGBoost Error vs Moneyness', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "XGBoost_Error_vs_Log-Moneyness.png", dpi=120)
plt.show()

In [ ]:
test_analysis['Moneyness_Type'] = pd.cut(
    test_analysis['LOG_MONEYNESS'], 
    bins=[-np.inf, -0.05, 0.05, np.inf],
    labels=['OTM', 'ATM', 'ITM']
)

test_analysis['Maturity_Type'] = pd.cut(
    test_analysis['DTE'],
    bins=[0, 30, 90, np.inf],
    labels=['Short (0-30d)', 'Medium (30-90d)', 'Long (90d+)']
)

pivot_error = test_analysis.groupby(['Moneyness_Type', 'Maturity_Type'])['ABS_ERROR'].mean().unstack()
pivot_count = test_analysis.groupby(['Moneyness_Type', 'Maturity_Type']).size().unstack()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(pivot_error, annot=True, fmt='.2f', cmap='RdYlGn_r', 
            ax=axes[0], cbar_kws={'label': 'MAE (USD)'}, vmin=0, vmax=15)
axes[0].set_title('Mean Absolute Error by Option Type', fontweight='bold')
axes[0].set_xlabel('Maturity')
axes[0].set_ylabel('Moneyness')

sns.heatmap(pivot_count, annot=True, fmt='d', cmap='Blues', 
            ax=axes[1], cbar_kws={'label': 'Sample Count'})
axes[1].set_title('Sample Count by Option Type', fontweight='bold')
axes[1].set_xlabel('Maturity')
axes[1].set_ylabel('Moneyness')

plt.tight_layout()
plt.savefig(FIGS_DIR / "prediction_quality_by_option_type.png", dpi=120)
plt.show()

## 13. Feature Importance

In [ ]:
rf_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_best.feature_importances_,
    'Model': 'Random Forest'
})

xgb_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': xgb_final.feature_importances_,
    'Model': 'XGBoost'
})

importance_df = pd.concat([rf_importance, xgb_importance])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

xgb_imp_sorted = xgb_importance.sort_values('Importance', ascending=True)
axes[0].barh(xgb_imp_sorted['Feature'], xgb_imp_sorted['Importance'], color='seagreen', edgecolor='black')
axes[0].set_xlabel('Importance Score')
axes[0].set_title('XGBoost Feature Importance', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

rf_imp_sorted = rf_importance.sort_values('Importance', ascending=True)
axes[1].barh(rf_imp_sorted['Feature'], rf_imp_sorted['Importance'], color='steelblue', edgecolor='black')
axes[1].set_xlabel('Importance Score')
axes[1].set_title('Random Forest Feature Importance', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(FIGS_DIR / "feature_importance_comparison.png", dpi=120)
plt.show()

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(xgb_imp_sorted['Feature'], xgb_imp_sorted['Importance'], color='teal', edgecolor='black')
ax.set_xlabel('Importance Score')
ax.set_title('XGBoost Feature Importance', fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "feature_importance_xgboost.png", dpi=120)
plt.show()

## 14. Cross-Validation

In [ ]:
print("Running 5-fold cross-validation on XGBoost...")

CV_SAMPLE_SIZE = 100000
X_cv_sample = X_train.sample(n=CV_SAMPLE_SIZE, random_state=42)
y_cv_sample = y_train.loc[X_cv_sample.index]

cv_params = xgb_grid_search.best_params_.copy()
if 'n_estimators' in cv_params:
    del cv_params['n_estimators']

xgb_cv_model = XGBRegressor(
    **cv_params,
    n_estimators=300,
    n_jobs=-1,
    random_state=42
)

cv_scores_mae = cross_val_score(xgb_cv_model, X_cv_sample, y_cv_sample, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
cv_scores_r2 = cross_val_score(xgb_cv_model, X_cv_sample, y_cv_sample, cv=5, scoring='r2', n_jobs=-1)

cv_mae = -cv_scores_mae

print("\nCross-Validation Results (5 folds on 100k sample):")
print("="*50)
print(f"MAE per fold: {cv_mae.round(2)}")
print(f"Mean MAE: {cv_mae.mean():.2f} ± {cv_mae.std():.2f} USD")
print(f"\nR² per fold: {cv_scores_r2.round(4)}")
print(f"Mean R²: {cv_scores_r2.mean():.4f} ± {cv_scores_r2.std():.4f}")

## 15. Save Best Model

In [ ]:
model_path = MODELS_DIR / "xgboost_best_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(xgb_final, f)
print(f"Model saved to: {model_path}")

config = {
    'model_type': 'XGBoost',
    'features': feature_cols,
    'target': 'C_LAST',
    'best_params': xgb_grid_search.best_params_,
    'best_iteration': int(xgb_final.best_iteration),
    'test_performance': {
        'MAE': float(xgb_test_metrics['MAE']),
        'RMSE': float(xgb_test_metrics['RMSE']),
        'R2': float(xgb_test_metrics['R2'])
    },
    'cv_performance': {
        'MAE_mean': float(cv_mae.mean()),
        'MAE_std': float(cv_mae.std()),
        'R2_mean': float(cv_scores_r2.mean()),
        'R2_std': float(cv_scores_r2.std())
    }
}

config_path = MODELS_DIR / "model_config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
print(f"Configuration saved to: {config_path}")

print("\nModel Configuration:")
print(json.dumps(config, indent=2))

## 16. Summary and Conclusions

### Key Results

| Model | MAE (USD) | RMSE (USD) | R² |
|-------|-----------|------------|----|
| Linear Regression | ~16.31 | ~25.41 | ~0.62 |
| Random Forest (tuned) | ~6.50 | ~15.78 | ~0.85 |
| **XGBoost (tuned)** | **~6.09** | **~14.73** | **~0.87** |

### Main Findings

1. **Non-linear models significantly outperform linear regression**
   - XGBoost achieves 63% reduction in MAE vs Linear Regression
   - Confirms option pricing is fundamentally a non-linear problem

2. **Grid Search improved model performance**
   - Systematic hyperparameter tuning identified optimal configurations
   - Best XGBoost params: learning_rate, max_depth, n_estimators tuned

3. **Most important features (aligned with financial theory):**
   - LOG_MONEYNESS: Relative strike position
   - UNDERLYING_LAST: Stock price (intrinsic value)
   - C_IV: Implied volatility
   - DTE: Time to expiration

4. **Model behavior:**
   - Best accuracy for ATM options (most liquid)
   - Higher errors for deep ITM (larger absolute prices)
   - Stable across cross-validation folds

5. **PCA Analysis:**
   - All 6 features contribute meaningful variance
   - Dimensionality reduction not recommended

### Limitations

- Model trained on historical data (2016-2020)
- Performance may degrade in extreme market conditions
- Greeks excluded to avoid data leakage